Azure AI Inference Script using GitHub Models
Free to use in GitHub Codespaces with your GitHub token

# Check if token and good endpoibt

In [1]:
import os
import requests

token = os.environ.get("GITHUB_TOKEN")
print(f"Token type: {'Classic' if token.startswith('ghp_') else 'Fine-grained'}")

response = requests.post(
    "https://models.inference.ai.azure.com/chat/completions",
    headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
    json={"model": "gpt-4o-mini", "messages": [{"role": "user", "content": "Hello"}]}
)

print(f"Status: {response.status_code}")
if response.status_code == 200:
    print("✅ Ça marche!")

Token type: Classic
Status: 200
✅ Ça marche!


In [3]:
#!/usr/bin/env python3
"""
Azure AI Inference Script avec gestion correcte des interruptions
Utilisez Ctrl+C pour interrompre à tout moment
"""

import os
import sys
import signal
import time
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

def signal_handler(signum, frame):
    """Gestionnaire pour Ctrl+C"""
    print("\n\n👋 Interruption détectée. Au revoir!")
    sys.exit(0)

def setup_client():
    """Setup the Azure AI client with GitHub token"""
    # Enregistrer le gestionnaire de signal
    signal.signal(signal.SIGINT, signal_handler)
    
    # Try to get token from environment variable
    token = os.environ.get("GITHUB_TOKEN")
    
    if not token:
        # If not in env, try to get it from Codespace secrets
        token = os.environ.get("CODESPACES_SECRET_GITHUB_TOKEN")
    
    if not token:
        print("ERROR: GITHUB_TOKEN not found!")
        print("Please set it using one of these methods:")
        print("1. In Codespaces: Settings > Secrets > New secret")
        print("2. Or run: export GITHUB_TOKEN='your-token-here'")
        sys.exit(1)
    
    endpoint = "https://models.inference.ai.azure.com"
    
    client = ChatCompletionsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token),
    )
    
    return client

def get_completion(prompt: str,
                   client, 
                   model_name="gpt-4o-mini", 
                   temperature=1.0, 
                   max_tokens=1000, 
                   top_p=1.0,
                   system_prompt: str = "You are a helpful assistant."
) -> str:
    """
    Get completion from the AI model
    Note: Using gpt-4o-mini by default as it's more cost-effective
    Generate a text completion using an OpenAI-compatible client.

    Args:
        prompt (str): User input or question.
        client: OpenAI client (e.g., openai.OpenAI()).
        model_name (str): Model name to use (default: gpt-4o-mini).
        temperature (float): Controls creativity (0.0–2.0).
        max_tokens (int): Maximum number of tokens to generate.
        top_p (float): Nucleus sampling parameter (0.0–1.0).
        system_prompt (str): Optional system instruction.

    Returns:
        str: The model's text response.
    """
    try:
        response = client.complete(
            messages=[
                SystemMessage(content=system_prompt),
                UserMessage(content=prompt),
            ],
            model=model_name,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p
        )
        return response.choices[0].message.content
    except KeyboardInterrupt:
        raise  # Re-raise pour que le main puisse le gérer
    except Exception as e:
        return f"Error: {str(e)}"
    
def main():
    """Main execution function avec gestion des interruptions"""
    try:
        # Setup client
        client = setup_client()
        
        # Available models in GitHub Models (free tier)
        print("\n" + "="*60)
        print("🤖 GITHUB MODELS AI CHAT")
        print("="*60)
        print("\nModèles disponibles:")
        print("  • gpt-4o-mini (recommandé - plus de requêtes)")
        print("  • gpt-4o (puissant mais limité)")
        print("  • Phi-3-medium-128k-instruct")
        print("  • Llama-3.2-90B-Vision-Instruct")
        print("\n💡 Astuce: Utilisez Ctrl+C à tout moment pour quitter")
        print("-"*60)
        
        # Choose model
        model_name = "gpt-4o-mini"
        print(f"\nModèle sélectionné: {model_name}")
        
        # Example 1: Simple completion
        print("\n" + "="*60)
        print("📝 Test rapide")
        print("="*60)
        
        text = "oh say can you see"
        prompt = f"Complete this text and explain what it is: '{text}'"
        
        print(f"Prompt: {prompt}")
        print("\n🤔 Réflexion en cours...")
        
        response = get_completion(prompt, client, model_name)
        print(f"\n🤖 Réponse: {response}")
        
        # Interactive mode
        print("\n" + "="*60)
        print("💬 MODE INTERACTIF")
        print("="*60)
        print("• Tapez votre message et appuyez sur Entrée")
        print("• Tapez 'quit', 'exit' ou 'q' pour quitter")
        print("• Utilisez Ctrl+C pour interrompre à tout moment")
        print("-"*60)
        
        while True:
            try:
                # Prompt avec gestion d'interruption
                user_input = input("\n💭 Vous: (use q for exit)")
                
                # Vérifier les commandes de sortie
                if user_input.lower() in ['quit', 'exit', 'q']:
                    print("\n👋 Au revoir!")
                    break
                
                # Vérifier si l'entrée est vide
                if not user_input.strip():
                    print("⚠️  Veuillez entrer un message.")
                    continue
                
                # Indicateur de traitement
                print("\n🤖 IA: ", end="", flush=True)
                print("(en cours...)", end="", flush=True)
                
                # Obtenir la réponse
                response = get_completion(user_input, client, model_name)
                
                # Effacer "(en cours...)" et afficher la réponse
                print("\r🤖 IA: " + " " * 20, end="\r")  # Effacer la ligne
                print(f"🤖 IA: {response}")
                
            except EOFError:
                # Gestion de Ctrl+D (fin de fichier)
                print("\n\n👋 Fin de session détectée. Au revoir!")
                break
            except KeyboardInterrupt:
                # Gestion de Ctrl+C pendant l'input
                print("\n\n⚠️  Interruption détectée.")
                confirm = input("Voulez-vous vraiment quitter? (o/n): ")
                if confirm.lower() in ['o', 'oui', 'y', 'yes']:
                    print("👋 Au revoir!")
                    break
                else:
                    print("Reprise du chat...")
                    continue
                
    except KeyboardInterrupt:
        # Gestion globale de Ctrl+C
        print("\n\n👋 Programme interrompu. Au revoir!")
        sys.exit(0)
    except Exception as e:
        print(f"\n\n❌ Erreur inattendue: {e}")
        sys.exit(1)

if __name__ == "__main__":
    # Protection supplémentaire pour les interruptions
    try:
        main()
    except KeyboardInterrupt:
        print("\n\n👋 Arrêt du programme.")
        sys.exit(0)
    except Exception as e:
        print(f"\n❌ Erreur fatale: {e}")
        sys.exit(1)


🤖 GITHUB MODELS AI CHAT

Modèles disponibles:
  • gpt-4o-mini (recommandé - plus de requêtes)
  • gpt-4o (puissant mais limité)
  • Phi-3-medium-128k-instruct
  • Llama-3.2-90B-Vision-Instruct

💡 Astuce: Utilisez Ctrl+C à tout moment pour quitter
------------------------------------------------------------

Modèle sélectionné: gpt-4o-mini

📝 Test rapide
Prompt: Complete this text and explain what it is: 'oh say can you see'

🤔 Réflexion en cours...

🤖 Réponse: The text "Oh say can you see" is the opening line of the national anthem of the United States, titled "The Star-Spangled Banner." The song was written by Francis Scott Key in 1814. Key was inspired to write the poem, which later became the anthem, after witnessing the bombardment of Fort McHenry during the War of 1812 and seeing the American flag still flying over the fort the next morning.

The anthem reflects themes of patriotism, resilience, and the struggle for freedom. It is often performed at public events, especially spor